In [163]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, count, date_format


import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [164]:
spark = SparkSession.builder \
    .appName("SPARK TRAB - QUESTION 2 A") \
    .getOrCreate()

In [165]:
df_orig = spark.read.json("./data/eiffel-tower-reviews.json")

In [166]:
df_orig.show()

+--------------------+--------------------+-----------+--------------------+------------+------------+--------------------+--------------------+
|                 _id|              author|bubbleCount|         collectedAt|   createdAt|       query|                text|               title|
+--------------------+--------------------+-----------+--------------------+------------+------------+--------------------+--------------------+
|{5921cdae4b679c46...|{0, 0, Since this...|         50|{2017-05-21T17:26...|May 20, 2017|Eiffel_Tower|This is the most ...|Must do even it w...|
|{5921cdae4b679c46...|{10, 4, Since Aug...|         50|{2017-05-21T17:26...|May 20, 2017|Eiffel_Tower|My significant ot...|           A Classic|
|{5921cdae4b679c46...|{9, 4, Since Nov ...|         50|{2017-05-21T17:26...|May 20, 2017|Eiffel_Tower|We had a tour to ...|         Wet weather|
|{5921cdae4b679c46...|{16, 5, Since Apr...|         50|{2017-05-21T17:26...|May 20, 2017|Eiffel_Tower|Visited with my w...|    Not

In [167]:
df_orig.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- author: struct (nullable = true)
 |    |-- helpfulVotes: string (nullable = true)
 |    |-- level: string (nullable = true)
 |    |-- memberSince: string (nullable = true)
 |    |-- postForum: string (nullable = true)
 |    |-- ratings: string (nullable = true)
 |    |-- reviews: string (nullable = true)
 |-- bubbleCount: long (nullable = true)
 |-- collectedAt: struct (nullable = true)
 |    |-- $date: string (nullable = true)
 |-- createdAt: string (nullable = true)
 |-- query: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)


In [168]:
date = df_orig.select("createdAt").dropna()

date.groupBy("createdAt").agg(count("createdAt").alias("count")).orderBy(col("count").desc()).show()

+------------------+-----+
|         createdAt|count|
+------------------+-----+
|     July 25, 2016|   48|
|September 28, 2015|   46|
|   August 13, 2015|   37|
|    April 10, 2017|   35|
| September 8, 2016|   35|
|   August 14, 2015|   34|
|   August 10, 2016|   33|
|  October 17, 2016|   32|
|September 15, 2015|   32|
|  November 1, 2016|   31|
|  October 18, 2016|   29|
|    April 25, 2016|   27|
|    August 7, 2016|   27|
|  February 8, 2016|   27|
|      May 27, 2016|   27|
|     June 30, 2016|   27|
|      June 8, 2016|   26|
| September 2, 2015|   26|
|   August 23, 2016|   26|
|   August 18, 2015|   26|
+------------------+-----+


In [169]:
date = date.withColumn("date", to_date(date["createdAt"], "MMMM d, yyyy"))

date = date.withColumn("month_year", date_format("date", "MM/yy"))

date_count = date.groupBy("date").agg(count("date").alias("count"))

date_count2 = date.groupBy("month_year").agg(count("month_year").alias("count"))

date_count.show()

+----------+-----+
|      date|count|
+----------+-----+
|2016-03-01|   14|
|2017-01-06|   10|
|2016-04-25|   27|
|2016-10-03|   14|
|2016-08-31|    5|
|2016-08-15|    7|
|2016-07-26|   26|
|2016-05-03|   13|
|2015-12-22|   12|
|2015-09-02|   26|
|2017-02-26|    3|
|2016-01-28|   11|
|2017-01-24|   12|
|2016-12-19|    1|
|2016-11-08|    1|
|2016-07-17|    5|
|2015-09-28|   46|
|2016-08-23|   26|
|2016-07-03|   19|
|2017-02-16|    9|
+----------+-----+


In [170]:
date_count_pd = date_count.toPandas()

date_count_pd2 = date_count2.toPandas()

In [171]:
sns.set(style="darkgrid", rc={
    "axes.facecolor": "#252434",
    "figure.facecolor": "#202124",
    "grid.color": "#37474f",
    "axes.edgecolor": "#202124",
    "font.size": 16,          
    "axes.labelsize": 20,     
    "axes.titlesize": 24,     
    "xtick.labelsize": 16,   
    "ytick.labelsize": 16  
})

In [172]:
plt.figure(figsize=(28, 12), facecolor='#202124')
sns.lineplot(x='date', y='count', data=date_count_pd, color='#ff3d2b', linewidth=2.5)

plt.xlabel('Data', color='#ffffff', fontsize=22)  
plt.ylabel('Número de Reviews', color='#ffffff', fontsize=22)  
plt.title('Distribuição Temporal de Reviews - Torre Eiffel', color='#ffffff', fontsize=28)  

plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))

plt.xticks(color='#ffffff', fontsize=18) 
plt.yticks(color='#ffffff', fontsize=18) 

plt.tight_layout(pad=3.0)
plt.savefig('./outcome/Q2/temporal_distribution.pdf', facecolor='#202124', bbox_inches='tight')
plt.close()

In [173]:
plt.figure(figsize=(28, 12), facecolor='#202124')
sns.lineplot(x='month_year', y='count', data=date_count_pd2, color='#ff3d2b', marker= "o", linewidth=2.5)

for x, y in zip(date_count_pd2['month_year'], date_count_pd2['count']):
    plt.annotate(f'{y}', (x, y), textcoords="offset points", xytext=(0,10), ha='center', color='white', fontsize=16, rotation=-25)

plt.xlabel('Data Agrupada por mês', color='#ffffff', fontsize=22)  
plt.ylabel('Número de Reviews', color='#ffffff', fontsize=22)  
plt.title('Distribuição Temporal de Reviews - Torre Eiffel', color='#ffffff', fontsize=28)  

plt.xticks(color='#ffffff', fontsize=18) 
plt.yticks(color='#ffffff', fontsize=18) 

plt.tight_layout(pad=3.0)
plt.savefig('./outcome/Q2/temporal_distribution2.pdf', facecolor='#202124', bbox_inches='tight')
plt.close()